In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

 
driver.get("https://www.bbc.com/news")
print(driver.title)

Home - BBC News


In [3]:
news_urls=[]
def get_urls():
 elements_for_url=driver.find_elements(By.CLASS_NAME, 'gs-c-promo-heading')
 for element in elements_for_url:
        news_urls.append(element.get_attribute('href'))
 print(len(elements_for_url))
get_urls()
print(len(news_urls))#list with all the urls to the articels. should be equally long to elements_for_url
print(news_urls[12])

75
75
https://www.bbc.com/news/business-67294106


In [4]:
#here we extract all the text contents from the articles. This takes approximately 2 minutes with a total of 76 articels today
def extract_contents(text_contents):
  try:
   article = driver.find_element(By.TAG_NAME, 'article')
   text_contents.append(article.text)
  except: #exception if no article element is there. doesn´t happen often, but has to be handled
   article = driver.find_element(By.TAG_NAME, 'body')
   text_contents.append(article.text)
   print("article not found") #in this test, 3 of 73 articels had no article element. 
#extract_contents()
#print(len(text_contents)) 

In [5]:

#here are a few exceptions, because the bbc articles have different html structures
def get_headers(headers):
  try:
   header=driver.find_element(By.ID, 'main-heading')
   headers.append(header.text)
  except:
   try:
     header = driver.find_element(By.CLASS_NAME, 'qa-story-headline')
     headers.append(header.text)
   except:
    try:
     header = driver.find_element(By.CLASS_NAME, 'article-headline__text')
     headers.append(header.text)
    except:
     header='unknown'
     headers.append(header)
     print('no header')
#get_headers()




In [6]:
def get_timestamps(time):
  try:
   time_element=driver.find_element(By.TAG_NAME, 'time')
   date=(time_element.get_attribute('datetime'))
   time.append(date)
  except:
    date='unknown'
    time.append(date)
    print('no date')

In [7]:
#main method to extract all contents except the authors(might follow later). Runs way faster than the different smaller methods before
text_contents=[]
time=[]
headers=[]
def extract_all():
    for i in range(len(news_urls)):
       driver.get(news_urls[i]) #we go to the specific article by using the url
       extract_contents(text_contents)
       get_timestamps(time)
       get_headers(headers)
    for i in range(len(time)):
      if time[i]==None:
       time[i]='unknown'
extract_all()
print(text_contents[10])
print(time[10])
print(headers[10])

no header
article not found
no header
article not found
no date
no header
no date
no date
no date
no date
no date
no date
no date
no header
no date
no header
Emily Hampshire: Actress apologises for Depp and Heard costume
Published
1 hour ago
Share
IMAGE SOURCE,
GETTY IMAGES
By Natasha Preskey
BBC News
Schitt's Creek actress Emily Hampshire has apologised for dressing up as Johnny Depp and Amber Heard with a friend for Halloween.
Hampshire, 42, told her Instagram followers she was "sorry and ashamed".
In a now-deleted post, Hampshire was seen styled as Depp, with drawn-on facial hair and tattoos, while her friend screwed up her face in mock distress as the actor's ex-wife.
Former couple Depp and Heard both accused each other of domestic abuse.
Hampshire, who played Stevie Budd in sitcom Schitt's Creek, described the costume as "one of the most thoughtless, insensitive, and ignorant things" she had ever done.
In the original post, the Canadian actress was seen wearing a pin-striped suit,

In [8]:
for item in headers:
  print(item)
print(len(headers))
counter=0
for i in range(len(headers)):
    if headers[i]=='unknown':
      counter=counter+1
print(counter)
for item in news_urls:
  print(item)

unknown
Israel Gaza: Children must be off limits, says father of abducted kids
'I simply don’t want to die at 24' - Gaza woman trapped at Rafah crossing
Could an Israeli ground invasion of Gaza meet its aims?
Gaza tunnels: Breaking down videos from Hamas' secret network
What China wants from Israel-Hamas war
Suspected mushroom poisoning: Australian woman charged with three murders
Baby monitor captures moment window is blown in by Storm Ciarán
Li Keqiang: Official nerves show as BBC hears praise for dead Chinese leader
Swiss village of Aarwangen in ding-dong over challenge to cowbells
Emily Hampshire: Actress apologises for Depp and Heard costume
Donald Trump Jr's courtroom debut: Chuckles, smiles and mispronunciation
Boss of Indonesia cough syrup maker jailed after child deaths
Shubhneet Singh: Punjabi rapper faces heat over concert hoodie
Taylor Tomlinson: Comedian, 29, fills James Corden's late TV show slot
Olivia Rodrigo 'beyond excited' her song is in Hunger Games film
Taylor Toml

In [9]:
def get_authors(article):
    authors = []
    lines = article.split('\n')
    for line in lines[:13]:  # Loop through lines from lines[0] to lines[10]
        if "By" in line:
            authors.append(line)
    return authors

all_authors = []
def get_all_authors():
    for article in text_contents:
        authors = get_authors(article)
        if not authors:  # Check if authors is an empty list
            all_authors.append("no author")
        else:
            all_authors.extend(authors)

# Print the authors
for author in all_authors:
    print(author)


get_all_authors()
print(all_authors[10])
print(len(all_authors))
print(all_authors)

counter=0
for i in range(len(all_authors)):
    if all_authors[i]=='no author':
      counter=counter+1
print(counter)

#<p class="lx-commentary__meta-reporter gel-long-primer" data-reactid=".18n36quudgy.0.2.0.0.1.0.1.0.$lx-tabs0.0.$lx-commentary.$lx-commentary.0.2.0.0">  Edited by Nadia Ragozhina and Samuel Horti</p>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By Jeremy Bowen</div>
#<div class="ssrcss-68pt20-Text-TextContributorName e8mq1e96">By David Gritten</div>

By Natasha Preskey
75
['no author', 'By Jon Donnison', 'By Fergal Keane', 'By Paul Kirby', 'no author', 'By Tessa Wong', 'By Hannah Ritchie and Simon Fraser', 'no author', 'By Stephen McDonell', 'By Imogen Foulkes', 'By Natasha Preskey', 'By Kayla Epstein', 'By Peter Hoskins', 'By Cherylann Mollan', 'By Annabel Rackham', 'By Helen Bushby', 'By Annabel Rackham', 'By Helen Bushby', 'By Brandon Drenon', 'By Wycliffe Muia', 'By Michelle Roberts', 'By James Oliver, Will Dahlgreen, James Melley & Andy Verity', 'no author', 'no author', 'By Paul Glynn', 'By Jewel Kiriungi', 'By Nick Thorpe', 'By Jessica Parker & Danai Nesta Kupemba', 'By MaryLou Costa', 'no author', 'no author', 'no author', 'no author', 'no author', 'By Shivani Chaudhari and Sonia Watson', 'no author', 'By Mark Savage', 'no author', 'By Andre Rhoden-Paul', 'By Chloe Kim', 'no author', 'By Hannah Ritchie and Simon Fraser', 'By Natasha Preskey', 'By Kayla Epstein', 'By James Oliver, Will Dahlgreen, James Melley & Andy Verity',

In [10]:
article1 = {
  "header": headers[10],
  "url": news_urls[10],
  "timestamp":time[10],
  "author":all_authors[10],
  "text_content":text_contents[10]
}

print(article1)




{'header': 'Emily Hampshire: Actress apologises for Depp and Heard costume', 'url': 'https://www.bbc.com/news/entertainment-arts-67296157', 'timestamp': '2023-11-02T11:04:17.000Z', 'author': 'By Natasha Preskey', 'text_content': 'Emily Hampshire: Actress apologises for Depp and Heard costume\nPublished\n1 hour ago\nShare\nIMAGE SOURCE,\nGETTY IMAGES\nBy Natasha Preskey\nBBC News\nSchitt\'s Creek actress Emily Hampshire has apologised for dressing up as Johnny Depp and Amber Heard with a friend for Halloween.\nHampshire, 42, told her Instagram followers she was "sorry and ashamed".\nIn a now-deleted post, Hampshire was seen styled as Depp, with drawn-on facial hair and tattoos, while her friend screwed up her face in mock distress as the actor\'s ex-wife.\nFormer couple Depp and Heard both accused each other of domestic abuse.\nHampshire, who played Stevie Budd in sitcom Schitt\'s Creek, described the costume as "one of the most thoughtless, insensitive, and ignorant things" she had eve

In [11]:
#dictionary with all infos, ordered after attributes
articles_info={}
for i in range(len(news_urls)):
 article_info_i={ 
  "headline": headers[i],
  "url": news_urls[i],
  "date":time[i],
  "authors":all_authors[i],
  "read-time":"no info",
  "ImageURL":"not yet",
  "ImageDescription":"not yet",
  "contents":text_contents[i],
  "scrapingTimeStamp":"not yet"
 }
 articles_info[i]=article_info_i
print(articles_info[0])
#URLId, Headline, Contents, Authors, UploadDate, ReadTime, ImageURL, ImageDescription

{'headline': 'unknown', 'url': 'https://www.bbc.com/news/live/world-middle-east-67281166', 'date': 'unknown', 'authors': 'no author', 'read-time': 'no info', 'ImageURL': 'not yet', 'ImageDescription': 'not yet', 'contents': 'Posted at 12:17\n12:17\nWatch: \'I want to pass, we are not animals\'\nVideo content\nVideo caption: Rafah crossing: \'We\'ve seen death with our own eyes\'\nRafah crossing: \'We\'ve seen death with our own eyes\'\nWe\'ve just brought you the news that more people are today reported to have used the Rafah crossing to leave Gaza for Egypt.\nThe footage which reached us earlier from the Reuters news agency gave a sense of people\'s mood there - with one despondent Egyptian woman saying: "We\'ve seen death with our own eyes... I want to pass, we are not animals."\nArticle share tools\nShare\nView more share options\nShare this post\nCopy this link\nRead more about these links.'}


In [12]:

print(type(articles_info[8]))


<class 'dict'>


In [13]:
import psycopg2

In [14]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [15]:
for article in articles_info.values():
    try:
        cursor.execute('''INSERT INTO Articles(URLId, Headline, Contents, Authors, UploadDate, ReadTime, ImageURL, ImageDescription,scrapingTimeStamp) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s);''', 
                       (article["url"], article["headline"], article["contents"], article["authors"], article["date"], article["read-time"], article["ImageURL"], article["ImageDescription"], article["scrapingTimeStamp"])
                       )
    except psycopg2.IntegrityError as e:
        if "duplicate key value violates unique constraint" in str(e):
            print(f"Article with URLId {article['url']} already exists in the database.")




Article with URLId https://www.bbc.com/news/live/world-middle-east-67281166 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67292422 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67293048 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67084141 already exists in the database.
Article with URLId https://www.bbc.com/news/world-middle-east-67280010 already exists in the database.
Article with URLId https://www.bbc.com/news/world-asia-china-67237146 already exists in the database.
Article with URLId https://www.bbc.com/news/world-australia-67293752 already exists in the database.
Article with URLId https://www.bbc.com/news/uk-67295305 already exists in the database.
Article with URLId https://www.bbc.com/news/world-asia-china-67294136 already exists in the database.
Article with URLId https://www.bbc.com/news/world-europe-67296413 already exists in the